In [204]:
benchmarks = [
    {"name": "Amplitude Estimation (AE)", "id": "1", "filename": "ae"},
    {"name": "Deutsch-Jozsa", "id": "2", "filename": "dj"},
    {"name": "Graph State", "id": "3", "filename": "graphstate"},
    {"name": "GHZ State", "id": "4", "filename": "ghz"},
    {"name": "Grover's (no ancilla)", "id": "5", "filename": "grover-noancilla"},
    {"name": "Grover's (v-chain)", "id": "6", "filename": "grover-v-chain"},
    {
        "name": "Portfolio Optimization with QAOA",
        "id": "7",
        "filename": "portfolioqaoa",
    },
    {"name": "Portfolio Optimization with VQE", "id": "8", "filename": "portfoliovqe"},
    {
        "name": "Quantum Approximation Optimization Algorithm (QAOA)",
        "id": "9",
        "filename": "qaoa",
    },
    {"name": "Quantum Fourier Transformation (QFT)", "id": "10", "filename": "qft"},
    {"name": "QFT Entangled", "id": "11", "filename": "qftentangled"},
    {"name": "Quantum Generative Adversarial Network", "id": "12", "filename": "qgan"},
    {
        "name": "Quantum Phase Estimation (QPE) exact",
        "id": "13",
        "filename": "qpeexact",
    },
    {
        "name": "Quantum Phase Estimation (QPE) inexact",
        "id": "14",
        "filename": "qpeinexact",
    },
    {"name": "Quantum Walk (no ancilla)", "id": "15", "filename": "qwalk-noancilla"},
    {"name": "Quantum Walk (v-chain)", "id": "16", "filename": "qwalk-v-chain"},
    {"name": "Variational Quantum Eigensolver (VQE)", "id": "17", "filename": "vqe"},
    {
        "name": "Efficient SU2 ansatz with Random Parameters",
        "id": "18",
        "filename": "su2random",
    },
    {
        "name": "Real Amplitudes ansatz with Random Parameters",
        "id": "19",
        "filename": "realamprandom",
    },
    {
        "name": "Two Local ansatz with Random Parameters",
        "id": "20",
        "filename": "twolocalrandom",
    },
    {"name": "W-State", "id": "21", "filename": "wstate"},
]

nonscalable_benchmarks = [
    {"name": "Excited State", "id": "22", "filename": "excitedstate"},
    {"name": "Ground State", "id": "23", "filename": "groundstate"},
    {"name": "HHL", "id": "24", "filename": "hhl"},
    {"name": "Pricing Call Option", "id": "25", "filename": "pricingcall"},
    {"name": "Pricing Put Option", "id": "26", "filename": "pricingput"},
    {"name": "Routing", "id": "27", "filename": "routing"},
    {"name": "Shor's", "id": "28", "filename": "shor"},
    {"name": "Travelling Salesman", "id": "29", "filename": "tsp"},
]

In [205]:
def get_opt_level(filename: str):
    """Extracts the optimization level based on a filename.

    Keyword arguments:
    filename -- filename of a benchmark

    Return values:
    num -- optimization level
    """

    pat = re.compile(r"opt\d")
    m = pat.search(filename)
    if m:
        num = m.group()[-1:]
    else:
        num = -1
    return int(num)

In [206]:
def get_num_qubits(filename: str):
    """Extracts the number of qubits based on a filename.

    Keyword arguments:
    filename -- filename of a benchmark

    Return values:
    num -- optimization level
    """

    pat = re.compile(r"(\d+)\.")
    m = pat.search(filename)
    if m:
        num = m.group()[0:-1]
    else:
        num = -1
    return int(num)

In [207]:
def get_tket_settings(filename: str):
    if "line" in filename:
        return "line"
    elif "graph" in filename:
        return "graph"


def get_gate_set(filename: str):
    if "oqc" in filename:
        return "oqc"
    elif "ionq" in filename:
        return "ionq"
    elif "ibm" in filename:
        return "ibm"
    elif "rigetti" in filename:
        return "rigetti"


def get_target_device(filename: str):
    if "ibm_washington" in filename:
        return "ibm_washington"
    elif "ibm_montreal" in filename:
        return "ibm_montreal"
    elif "rigetti_aspen_m1" in filename:
        return "rigetti_aspen_m1"
    elif "ionq11" in filename:
        return "ionq11"
    elif "oqc_lucy" in filename:
        return "oqc_lucy"

In [208]:
def get_compiler_and_settings(filename: str):
    if "qiskit" in filename:
        return ("qiskit", get_opt_level(filename))
    elif "tket" in filename:
        return ("tket", get_tket_settings(filename))

In [209]:
def parse_data(filename: str):
    """Extracts the necessary information from a given filename.

    Keyword arguments:
    filename -- name of file

    Return values:
    parsed_data -- parsed data extracted from filename
    """
    benchmark = filename.split("_")[1].lower().split("/")[-1]
    num_qubits = get_num_qubits(filename)
    indep_flag = "indep" in filename
    nativegates_flag = "nativegates" in filename
    mapped_flag = "mapped" in filename
    compiler, compiler_settings = get_compiler_and_settings(filename)
    gate_set = get_gate_set(filename)
    target_device = get_target_device(filename)

    path = os.path.join(filename)
    parsed_data = [
        benchmark,
        num_qubits,
        indep_flag,
        nativegates_flag,
        mapped_flag,
        compiler,
        compiler_settings,
        gate_set,
        target_device,
        path,
    ]
    return parsed_data

In [213]:
from mqt.bench.utils import utils, qiskit_helper, tket_helper
import os
import re
import pandas as pd

rows_list = []
# for filename in os.listdir(qasm_path):
directory = (
    "/Users/nils/Desktop/qasm_output/qasm_output"  # utils.get_qasm_output_path()
)

for f in os.listdir(directory):
    parsed_data = parse_data(os.path.join(directory, f))
    rows_list.append(parsed_data)


colnames = [
    "benchmark",
    "num_qubits",
    "indep_flag",
    "nativegates_flag",
    "mapped_flag",
    "compiler",
    "compiler_settings",
    "gate_set",
    "target_device",
    "path",
]

database = pd.DataFrame(rows_list, columns=colnames)
database["num_qubits"] = database["num_qubits"].astype(int)
database["benchmark"] = database["benchmark"].astype(str)

database.shape

(36044, 10)

In [214]:
print(list(database.indep_flag.unique()))

[False, True]


In [215]:
def filterDatabase(filterCriteria, database):
    colnames = [
        "benchmark",
        "num_qubits",
        "indep_flag",
        "nativegates_flag",
        "mapped_flag",
        "compiler",
        "compiler_settings",
        "gate_set",
        "target_device",
        "path",
    ]
    db_filtered = pd.DataFrame(columns=colnames)
    if len(database) == 0:
        return []

    (
        (min_qubits, max_qubits),
        indices_benchmarks,
        (indep_qiskit_compiler, indep_tket_compiler),
        (
            (nativegates_qiskit_compiler, nativegates_tket_compiler),
            native_qiskit_opt_lvls,
            native_gatesets,
        ),
        (
            (mapped_qiskit_compiler, mapped_tket_compiler),
            (mapped_qiskit_opt_lvls, mapped_tket_placements),
            mapped_devices,
        ),
    ) = filterCriteria

    for identifier in indices_benchmarks:
        if int(identifier) > 0 and int(identifier) <= len(benchmarks):
            name = benchmarks[int(identifier) - 1]["filename"]

            db_tmp = database.loc[
                (database["num_qubits"] >= min_qubits)
                & (database["num_qubits"] <= max_qubits)
                & (database["benchmark"] == name)
            ]
        elif int(identifier) > 0 and int(identifier) <= len(benchmarks) + len(
            nonscalable_benchmarks
        ):
            name = nonscalable_benchmarks[int(identifier) - 1 - len(benchmarks)][
                "filename"
            ]
            db_tmp = database.loc[database["benchmark"] == name]
        else:
            print("FAIL")

        print(name, ":", len(db_tmp))

        if indep_qiskit_compiler:
            db_tmp1 = db_tmp.loc[
                (db_tmp["indep_flag"] == True) & (db_tmp["compiler"] == "qiskit")
            ]
            db_filtered = pd.concat([db_filtered, db_tmp1])

        if indep_tket_compiler:
            db_tmp2 = db_tmp.loc[
                (db_tmp["indep_flag"] == True) & (db_tmp["compiler"] == "tket")
            ]
            db_filtered = pd.concat([db_filtered, db_tmp2])

        if nativegates_qiskit_compiler:
            for gate_set in native_gatesets:
                for opt_lvl in native_qiskit_opt_lvls:
                    db_tmp3 = db_tmp.loc[
                        (db_tmp["nativegates_flag"] == True)
                        & (db_tmp["gate_set"] == gate_set)
                        & (db_tmp["compiler"] == "qiskit")
                        & (db_tmp["compiler_settings"] == opt_lvl)
                    ]
                    db_filtered = pd.concat([db_filtered, db_tmp3])

        if nativegates_tket_compiler:
            for gate_set in native_gatesets:
                db_tmp4 = db_tmp.loc[
                    (db_tmp["nativegates_flag"] == True)
                    & (db_tmp["gate_set"] == gate_set)
                    & (db_tmp["compiler"] == "tket")
                ]
                db_filtered = pd.concat([db_filtered, db_tmp4])

        if mapped_qiskit_compiler:
            for opt_lvl in native_qiskit_opt_lvls:
                for device in mapped_devices:
                    db_tmp5 = db_tmp.loc[
                        (db_tmp["mapped_flag"] == True)
                        & (db_tmp["target_device"] == device)
                        & (db_tmp["compiler"] == "qiskit")
                        & (db_tmp["compiler_settings"] == opt_lvl)
                    ]
                    db_filtered = pd.concat([db_filtered, db_tmp5])

        if mapped_tket_compiler:
            for placement in mapped_tket_placements:
                for device in mapped_devices:
                    db_tmp6 = db_tmp.loc[
                        (db_tmp["mapped_flag"] == True)
                        & (db_tmp["target_device"] == device)
                        & (db_tmp["compiler"] == "tket")
                        & (db_tmp["compiler_settings"] == placement)
                    ]
                    db_filtered = pd.concat([db_filtered, db_tmp6])

    return db_filtered["path"].to_list()

In [196]:
criteria = (
    (1, 130),
    [
        "1",
        "2",
        "3",
        "4",
        "5",
        "6",
        "7",
        "8",
        "9",
        "10",
        "11",
        "12",
        "13",
        "14",
        "15",
        "16",
        "17",
        "18",
        "19",
        "20",
        "21",
        "22",
        "23",
        "24",
        "25",
        "26",
        "27",
        "28",
        "29",
    ],
    (True, True),
    ((True, True), [0, 1, 2, 3], ["oqc", "ionq", "ibm", "rigetti"]),
    (
        (True, True),
        ([0, 1, 2, 3], ["graph", "line"]),
        ["ibm_washington", "rigetti_aspen_m1", "ionq11", "ibm_montreal", "oqc_lucy"],
    ),
)
paths = filterDatabase(criteria, database)

ae : 2035
dj : 4188
graphstate : 3971
ghz : 4270
grover-noancilla : 310
grover-v-chain : 0
portfolioqaoa : 352
portfoliovqe : 442
qaoa : 750
qft : 2060
qftentangled : 2009
qgan : 104
qpeexact : 2031
qpeinexact : 2029
qwalk-noancilla : 262
qwalk-v-chain : 0
vqe : 559
su2random : 1824
realamprandom : 1829
twolocalrandom : 1824
wstate : 4270
excitedstate : 0
groundstate : 0
hhl : 78
pricingcall : 343
pricingput : 341
routing : 103
shor : 8
tsp : 52


In [197]:
len(paths)

36044

In [198]:
print(
    database.loc[
        (database["nativegates_flag"] == True) & (database["compiler"] == "qiskit")
    ].shape,
    database.loc[
        (database["nativegates_flag"] == True) & (database["compiler"] == "tket")
    ].shape,
    database.loc[(database["nativegates_flag"] == True)].shape,
)

(17281, 10) (4038, 10) (21319, 10)


In [199]:
native_gatesets = ["oqc", "ionq", "ibm", "rigetti"]
native_qiskit_opt_lvls = [0, 1, 2, 3]
tmp = 0
for gate_set in native_gatesets:
    for opt_lvl in native_qiskit_opt_lvls:
        db_tmp3 = database.loc[
            (database["nativegates_flag"] == True)
            & (database["gate_set"] == gate_set)
            & (database["compiler"] == "qiskit")
            & (database["compiler_settings"] == opt_lvl)
        ]
        tmp += db_tmp3.shape[0]
print(tmp)

17281


In [200]:
native_gatesets = ["oqc", "ionq", "ibm", "rigetti"]
tmp = 0
for gate_set in native_gatesets:
    db_tmp3 = database.loc[
        (database["nativegates_flag"] == True)
        & (database["gate_set"] == gate_set)
        & (database["compiler"] == "tket")
    ]
    tmp += db_tmp3.shape[0]
print(tmp)

4038


In [201]:
database.loc[
    (database["nativegates_flag"] == True) & (database["compiler"] == "tket")
].gate_set.unique()

array(['ionq', 'oqc', 'rigetti', 'ibm'], dtype=object)

In [56]:
from zipfile import ZipFile, ZIP_DEFLATED


def create_zip_file():
    zf = zipfile.ZipFile("MQTBench_all.zip", "w")
    for dirname, subdirs, files in os.walk(utils.get_qasm_output_path()):
        for filename in files:
            zf.write(
                os.path.join(dirname, filename),
                compress_type=ZIP_DEFLATED,
                compresslevel=3,
                arcname=filename,
            )
    zf.close()

In [57]:
create_zip_file()